<a href="https://colab.research.google.com/github/unfortunate-code/Neural-Network-For-Taking-Derivatives/blob/main/Derivatives_LSTM_Big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install torchsummaryX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import itertools
import random
import re
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchsummaryX import summary

In [ ]:
def simplify_equation(line):
  x, y = line.strip().split('=')
  matches = re.match('d\((.*)\)/d(.*)', x)
  return matches.group(1), matches.group(2), y

In [ ]:
with open('derivatives-takehome/train.txt', 'r') as f:
  data = [simplify_equation(line) for line in f.readlines()]

In [ ]:
def tokenize_equations(eqn, var, var_map):
  if not var_map:
    var_map[var] = 'var0'
    var_index = 1
  else:
    var_index = int(sorted(list(var_map.values()))[-1][-1]) + 1
  curr = ''
  tokens = []
  for i in range(len(eqn)):
    if 'a' <= eqn[i] <='z' or 'A' <= eqn[i] <= 'Z':
      curr += eqn[i]
    else:
      if curr:
        if len(curr) == 1:
          if curr in var_map:
            tokens.append(var_map[curr])
          else:
            var_map[curr] = 'var' + str(var_index)
            var_index += 1
            tokens.append(var_map[curr])
        else:
          tokens.append(curr)
        curr = ''
      tokens.append(eqn[i])
  return tokens, var_map

In [ ]:
tokenized_data = []
for x, v, y in data:
  x_tokens, var_map = tokenize_equations(x, v, {})
  y_tokens, var_map = tokenize_equations(y, v, var_map)
  tokenized_data.append((x_tokens, y_tokens, {v: k for k, v in var_map.items()}))

In [ ]:
input_vocabulary = set(itertools.chain.from_iterable([x for x, y, _ in tokenized_data]))
output_vocabulary = set(itertools.chain.from_iterable([y for x, y, _ in tokenized_data]))

In [ ]:
variables = set(itertools.chain.from_iterable([z.values() for _, _, z in tokenized_data]))

In [ ]:
print(variables)
print(input_vocabulary)
print(output_vocabulary)
print(len(variables))
print(len(input_vocabulary))
print(len(output_vocabulary))

{'a', 's', 'c', 'r', 't', 'y', 'u', 'x', 'p', 'b', 'm', 'w', 'i', 'k', 'z', 'v', 'n', 'e', 'o'}
{'4', '1', '*', '(', '7', 'var1', '6', '3', '+', '8', '2', '^', 'exp', 'sin', 'var0', 'cos', '5', '0', '9', '-', ')'}
{'4', '1', '*', '(', '7', 'var1', '6', '3', '+', '2', '8', '^', 'exp', 'sin', 'var0', 'cos', '5', '0', '9', '-', ')'}
19
21
21


In [ ]:
start_token = '<SOS>'
end_token = '<EOS>'
input_vocabulary.add(start_token)
input_vocabulary.add(end_token)
output_vocabulary.add(start_token)
output_vocabulary.add(end_token)
input_token_to_index = {}
input_index_to_token = {}
index = 0
for token in input_vocabulary:
  input_token_to_index[token] = index
  input_index_to_token[index] = token
  index += 1
output_token_to_index = {}
output_index_to_token = {}
index = 0
for token in output_vocabulary:
  output_token_to_index[token] = index
  output_index_to_token[index] = token
  index += 1

In [ ]:
all_indices = set(range(1000000))
train_indices = set(random.sample(all_indices, 9 * len(all_indices) // 10))
all_indices -= train_indices
val_indices = set(random.sample(all_indices, len(all_indices) // 2))
all_indices -= val_indices
test_indices = all_indices

In [ ]:
print(len(train_indices))
print(len(test_indices))
print(len(val_indices))

900000
50000
50000


In [ ]:
train_data = [tokenized_data[i] for i in train_indices]
test_data = [tokenized_data[i] for i in test_indices]
val_data = [tokenized_data[i] for i in val_indices]

In [ ]:
class DerivativesDataset(Dataset):
  def __init__(self, data):
    self.data = data
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    x, y, map = self.data[index]
    x = torch.tensor([input_token_to_index[token] for token in x] + [input_token_to_index[end_token]])
    y = torch.tensor([output_token_to_index[token] for token in y] + [output_token_to_index[end_token]])
    return x, y, map

In [ ]:
class PadSequence:
  def __call__(self, batch):
    return torch.nn.utils.rnn.pad_sequence([x[0] for x in batch], batch_first=True, padding_value=input_token_to_index[end_token]), torch.nn.utils.rnn.pad_sequence([x[1] for x in batch], batch_first=True, padding_value=output_token_to_index[end_token]), [x[2] for x in batch]

In [ ]:
train_dataset = DerivativesDataset(train_data)
train_dataloader = DataLoader(train_dataset, batch_size=4096, shuffle=True, pin_memory=True, num_workers=2, collate_fn=PadSequence())

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size, layers, dropout):
    super(Encoder, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.layers = layers
    self.embedding_layer = nn.Embedding(input_size, hidden_size)
    self.dropout1 = nn.Dropout(dropout)
    self.linear1 = nn.Linear(hidden_size, 2 * hidden_size)
    self.linear2 = nn.Linear(hidden_size * 2, hidden_size)
    self.dropout2 = nn.Dropout(dropout)
    self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, num_layers=layers, batch_first=True)
    nn.init.kaiming_normal_(self.linear1.weight, mode='fan_in')
    nn.init.kaiming_normal_(self.linear2.weight, mode='fan_in')
  
  def forward(self, x):
    embeddings = self.embedding_layer(x)
    embeddings = self.dropout1(embeddings)
    embeddings = self.linear1(embeddings)
    embeddings = nn.functional.relu(embeddings)
    embeddings = self.linear2(embeddings)
    embeddings = nn.functional.relu(embeddings)
    embeddings = self.dropout2(embeddings)
    h0 = torch.zeros(self.layers, x.size(0), self.hidden_size).cuda()
    c0 = torch.zeros_like(h0).cuda()
    return self.lstm(embeddings, (h0, c0))

In [ ]:
class Decoder(nn.Module):
  def __init__(self, hidden_size, output_size, layers, dropout):
    super(Decoder, self).__init__()
    self.output_size = output_size
    self.hidden_size = hidden_size
    self.layers = layers
    self.embedding_layer = nn.Embedding(output_size, hidden_size)
    self.dropout1 = nn.Dropout(dropout)
    self.linear1 = nn.Linear(hidden_size, 2 * hidden_size)
    self.linear2 = nn.Linear(hidden_size * 2, hidden_size)
    self.dropout2 = nn.Dropout(dropout)
    self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, num_layers=layers, batch_first=True)
    self.fc_linear1 = nn.Linear(hidden_size, 2 * hidden_size)
    self.fc_linear2 = nn.Linear(2 * hidden_size, 4 * hidden_size)
    self.fc_linear3 = nn.Linear(4 * hidden_size, 2 * hidden_size)
    self.fc = nn.Sequential(self.fc_linear1,
                            nn.ReLU(),
                            self.fc_linear2,
                            nn.ReLU(),
                            self.fc_linear3,
                            nn.ReLU())
    self.out = nn.Linear(2 * hidden_size, output_size)
    nn.init.kaiming_normal_(self.linear1.weight, mode='fan_in')
    nn.init.kaiming_normal_(self.linear2.weight, mode='fan_in')
    nn.init.kaiming_normal_(self.fc_linear1.weight, mode='fan_in')
    nn.init.kaiming_normal_(self.fc_linear2.weight, mode='fan_in')
    nn.init.kaiming_normal_(self.fc_linear3.weight, mode='fan_in')

  def forward(self, x, h0, c0):
    embeddings = self.embedding_layer(x)
    embeddings = self.dropout1(embeddings)
    embeddings = self.linear1(embeddings)
    embeddings = nn.functional.relu(embeddings)
    embeddings = self.linear2(embeddings)
    embeddings = nn.functional.relu(embeddings)
    embeddings = self.dropout2(embeddings)
    hiddens, (hn, cn) = self.lstm(embeddings, (h0, c0))
    pred = self.fc(hn[-1])
    pred = self.out(pred)
    return pred, hn, cn

In [ ]:
hidden_size = 23
num_layers = 2

In [ ]:
encoder = Encoder(len(input_vocabulary), hidden_size, num_layers,0.2)
decoder = Decoder(len(output_vocabulary), hidden_size, num_layers, 0.2)
encoder = encoder.train().cuda()
decoder = decoder.train().cuda()

In [ ]:
print(encoder)


Encoder(
  (embedding_layer): Embedding(23, 23)
  (dropout1): Dropout(p=0.2, inplace=False)
  (linear1): Linear(in_features=23, out_features=46, bias=True)
  (linear2): Linear(in_features=46, out_features=23, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (lstm): LSTM(23, 23, num_layers=2, batch_first=True)
)


In [ ]:
print(decoder)

Decoder(
  (embedding_layer): Embedding(23, 23)
  (dropout1): Dropout(p=0.2, inplace=False)
  (linear1): Linear(in_features=23, out_features=46, bias=True)
  (linear2): Linear(in_features=46, out_features=23, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (lstm): LSTM(23, 23, num_layers=2, batch_first=True)
  (fc_linear1): Linear(in_features=23, out_features=46, bias=True)
  (fc_linear2): Linear(in_features=46, out_features=92, bias=True)
  (fc_linear3): Linear(in_features=92, out_features=46, bias=True)
  (fc): Sequential(
    (0): Linear(in_features=23, out_features=46, bias=True)
    (1): ReLU()
    (2): Linear(in_features=46, out_features=92, bias=True)
    (3): ReLU()
    (4): Linear(in_features=92, out_features=46, bias=True)
    (5): ReLU()
  )
  (out): Linear(in_features=46, out_features=23, bias=True)
)


In [ ]:
learning_rate = 0.001
epochs = 100
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(epochs):
  print('Epoch', epoch)
  total_loss = []
  for x, y, map in tqdm(train_dataloader):
    x = x.cuda()
    y = y.cuda()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    _, (enc_hn, enc_cn) = encoder(x)
    decoder_input = torch.IntTensor(len(x), 1).fill_(output_token_to_index[start_token]).cuda()
    dec_h, dec_c = enc_hn, enc_cn
    loss = 0
    for i in range(y.shape[1]):
      pred, dec_h, dec_c = decoder(decoder_input, dec_h, dec_c)
      decoder_input = y[:, i].unsqueeze(1)
      loss += criterion(pred, y[:, i])
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    total_loss.append(loss.item() / y.shape[1])
  print(sum(total_loss) / len(total_loss))

Epoch 0


100%|██████████| 220/220 [00:36<00:00,  6.00it/s]


1.6940062868803327
Epoch 1


100%|██████████| 220/220 [00:36<00:00,  6.03it/s]


0.799174789034704
Epoch 2


...

0.34070174782862733
Epoch 98


100%|██████████| 220/220 [00:36<00:00,  6.09it/s]


0.32605949406438706
Epoch 99


100%|██████████| 220/220 [00:37<00:00,  5.89it/s]

0.32693521063171704


In [ ]:
torch.save(encoder, 'encoder-large-hidden-epochs-100.model')
torch.save(decoder, 'decoder-large-hidden-epochs-100.model')

In [ ]:
def is_match(y, pred):
  for i in range(len(y)):
    if y[i] == output_token_to_index[end_token]: return True
    if i >= len(pred) or y[i] != pred[i]: return False
  return True

In [ ]:
def evaluate(dataloader, cutoff_length, encoder, decoder):
  encoder = encoder.eval()
  decoder = decoder.eval()
  count = correct_count = 0
  with torch.no_grad():
    for x, y, map in tqdm(dataloader):
      count += len(x)
      x, y = x.cuda(), y.cuda()
      _, (enc_hn, enc_cn) = encoder(x)
      for i in range(len(x)):
        prediction = []
        dec_h_i, dec_c_i = enc_hn[:, i, :].contiguous().unsqueeze(1), enc_cn[:, i, :].contiguous().unsqueeze(1)
        decoder_input = torch.IntTensor(1, 1).fill_(output_token_to_index[start_token]).cuda()
        decoded_indices = []
        for j in range(cutoff_length):
          pred, dec_h_i, dec_c_i = decoder(decoder_input, dec_h_i, dec_c_i)
          _, topi = pred.topk(1)
          topi = topi.item()
          
          if topi == output_token_to_index[end_token]:
            break
          else:
            decoded_indices.append(topi)
          decoder_input = torch.IntTensor(1, 1).fill_(topi).cuda()
        if is_match(y[i], decoded_indices): correct_count += 1
  return correct_count / count

In [ ]:
val_dataset = DerivativesDataset(val_data)
val_dataloader = DataLoader(val_dataset, batch_size=4096, shuffle=False, pin_memory=True, num_workers=2, collate_fn=PadSequence())
evaluate(val_dataloader, 40, encoder, decoder)

100%|██████████| 13/13 [09:40<00:00, 44.67s/it]


0.01214

In [ ]:
test_dataset = DerivativesDataset(test_data)
test_dataloader = DataLoader(test_dataset, batch_size=4096, shuffle=False, pin_memory=True, num_workers=2, collate_fn=PadSequence())
evaluate(test_dataloader, 40, encoder, decoder)

100%|██████████| 13/13 [09:39<00:00, 44.58s/it]


0.01164

In [ ]:
summary(encoder, test_dataset[0][0].cuda().unsqueeze(0))

                  Kernel Shape Output Shape  Params Mult-Adds
Layer                                                        
0_embedding_layer     [23, 23]  [1, 11, 23]   529.0     529.0
1_dropout1                   -  [1, 11, 23]       -         -
2_linear1             [23, 46]  [1, 11, 46]  1.104k    1.058k
3_linear2             [46, 23]  [1, 11, 23]  1.081k    1.058k
4_dropout2                   -  [1, 11, 23]       -         -
5_lstm                       -  [1, 11, 23]  8.832k    8.464k
--------------------------------------------------------------
                       Totals
Total params          11.546k
Trainable params      11.546k
Non-trainable params      0.0
Mult-Adds             11.109k


/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding_layer,"[23, 23]","[1, 11, 23]",529.0,529.0
1_dropout1,-,"[1, 11, 23]",NaN,NaN
2_linear1,"[23, 46]","[1, 11, 46]",1104.0,1058.0
3_linear2,"[46, 23]","[1, 11, 23]",1081.0,1058.0
4_dropout2,-,"[1, 11, 23]",NaN,NaN
5_lstm,-,"[1, 11, 23]",8832.0,8464.0
